# Measuring uptake of inhaled budesonide prescription guidance following COVID-19 infection

## Setup, data loading and prepreprocessing

In [1]:
import pandas as pd

In [2]:
## low count suppression function
def suppress(df_to_suppress,columns,n=10):
    for c in columns:
        if not c in df_to_suppress.columns:
            raise ValueError(f'column {c} not in dataframe')

    df = df_to_suppress
    for c in df.columns[df.columns.isin(columns)]:
        df[f'{c}'] = df[f'{c}'].astype(str)
        df[f'{c}'] = df[f'{c}'].apply(lambda x: '<10' if (float(x)>0 and float(x)<n ) else x)
    return df

Import output from cohort extractor with following exclusions already applied:

```
NOT has_died
AND registered
AND
(age_band = "65_plus" OR (age_band = "50_65" AND (primis_shield OR primis_nonshield))) 
AND
(sex = "M" OR sex = "F")
AND NOT covid_admission
AND NOT covid_emergency_admission
AND NOT has_previous_steroid_prescription
AND NOT primary_covid_hospital_admission
AND first_positive_test_type != "LFT_Only"
```

Expected row count is c.8000 (as of 2021-07-30)


In [4]:
df = pd.read_csv('../output/input.csv')

In [5]:
print(f'Total input row count: {len(df.index)}')

Total input row count: 1000


In [6]:
df.first_positive_test_date = pd.to_datetime(df.first_positive_test_date)
print(f'Most recent positive test date: {df.first_positive_test_date.max()}')

Most recent positive test date: 2021-12-31 00:00:00


Apply labels for three cohorts based on age at time of positive COVID-19 test and PRIMIS sheilding flags (estimated expected proportion):
* over 65s (85%)
* over 50s (15%)
  - with shielding flag (5%)
  - with non-shielding flag (10%)

In [7]:
df['cohort'] = df.apply(lambda x: x.age_band if x.age_band == '65_plus' else str(x.age_band) + ('H' if x.primis_shield==1 else 'L'),axis=1)
df_cohort_counts = df.groupby('cohort').count()['patient_id'].reset_index().rename(columns={'patient_id':'Patient count',"cohort":"Cohort"})
suppressed_df_cohort_counts = suppress(df_cohort_counts,['Patient count'])
suppressed_df_cohort_counts.style.hide_index()

Cohort,Patient count
50_65H,<10
50_65L,20
65_plus,42
lt_50H,39
lt_50L,397
nanH,54
nanL,446


## COVID-19 Test Types

Only patients with "PCR_Only" (expected ~82%) and "LFT_WithPCR" (18%) positive COVID-19 test results included

In [8]:
df_test_counts = df.groupby('first_positive_test_type').count()['patient_id'].reset_index().rename(columns={'patient_id':'Patient count',"first_positive_test_type":"Type of first recorded positive test"})
supressed_test_type_counts = suppress(df_test_counts,['Patient count'])
supressed_test_type_counts.style.hide_index()

Type of first recorded positive test,Patient count
LFT_WithPCR,58
PCR_Only,120


## Budesonide prescriptions by cohort

In [9]:
supressed_bd_prescriptions_all = suppress(df.groupby('budesonide_prescription').count()['patient_id'].reset_index().rename(columns={"budesonide_prescription":"Count of budesonide prescriptions","patient_id":"Count of patients"}),["Count of patients"])
supressed_bd_prescriptions_all['cohort'] = 'All'

supressed_bd_prescriptions_by_cohort = suppress(df.groupby(['cohort','budesonide_prescription']).count()['patient_id'].reset_index().rename(columns={"budesonide_prescription":"Count of budesonide prescriptions","patient_id":"Count of patients"}),['Count of patients'])
supressed_bd_prescriptions_by_cohort = pd.concat([supressed_bd_prescriptions_all,supressed_bd_prescriptions_by_cohort],ignore_index=True)
supressed_bd_prescriptions_by_cohort.style.hide_index()

Count of budesonide prescriptions,Count of patients,cohort
-2,<10,All
-1,10,All
0,737,All
1,165,All
2,71,All
3,15,All
0,<10,50_65H
0,14,50_65L
1,<10,50_65L
2,<10,50_65L


## Budesonide prescriptions by region

In [10]:
supressed_bd_prescriptions_by_region = suppress(df.groupby(['region','budesonide_prescription']).count()['patient_id'].reset_index().rename(columns={"budesonide_prescription":"Count of budesonide prescriptions","patient_id":"Count of patients"}),['Count of patients'])
supressed_bd_prescriptions_by_region = pd.concat([supressed_bd_prescriptions_all.rename(columns={'cohort':'region'}),supressed_bd_prescriptions_by_region],ignore_index=True)
supressed_bd_prescriptions_by_region.style.hide_index()

Count of budesonide prescriptions,Count of patients,region
-2,<10,All
-1,10,All
0,737,All
1,165,All
2,71,All
3,15,All
0,65,East Midlands
1,20,East Midlands
2,<10,East Midlands
3,<10,East Midlands


## Budesonide prescriptions by practice

In [11]:
supressed_bd_prescriptions_by_practice_id = suppress(df.groupby(['practice_id','budesonide_prescription']).count()['patient_id'].reset_index().rename(columns={"budesonide_prescription":"Count of budesonide prescriptions","patient_id":"Count of patients"}),['Count of patients'])
supressed_bd_prescriptions_by_practice_id = pd.concat([supressed_bd_prescriptions_all.rename(columns={'cohort':'practice_id'}),supressed_bd_prescriptions_by_practice_id],ignore_index=True)
supressed_bd_prescriptions_by_practice_id.style.hide_index()

Count of budesonide prescriptions,Count of patients,practice_id
-2,<10,All
-1,10,All
0,737,All
1,165,All
2,71,All
3,15,All
0,<10,11
1,<10,11
1,<10,12
0,<10,13


## Budesonide prescriptions by region, by practice

In [12]:
pd.set_option('display.max_rows', None)
region_practice_pivot = pd.pivot_table(df,index=['region','practice_id'],columns=['budesonide_prescription'],values='patient_id',aggfunc='count').rename(columns={"budesonide_prescription":"Count of budesonide prescriptions","patient_id":"Count of patients"})
region_practice_pivot.rename(columns=lambda x: str(x),inplace=True)
display(suppress(region_practice_pivot,list(region_practice_pivot.columns)))

budesonide_prescription                -2   -1     0    1    2    3
region                   practice_id                               
East Midlands            15           nan  nan   <10  nan  nan  nan
                         16           nan  nan   <10  <10  nan  nan
                         17           nan  nan   <10  <10  nan  nan
                         18           nan  nan   <10  <10  nan  nan
                         19           nan  nan   <10  <10  nan  nan
                         20           nan  nan   <10  <10  nan  <10
                         21           nan  nan   <10  <10  <10  nan
                         22           nan  nan   <10  <10  nan  nan
                         23           nan  nan   <10  <10  nan  nan
                         24           nan  nan   <10  <10  nan  nan
                         25           nan  nan   <10  nan  nan  nan
                         26           nan  nan   <10  <10  <10  nan
                         27           nan  nan   <10  <10  <10  nan
                         29           nan  nan   <10  <10  nan  nan
                         30           nan  nan   <10  nan  <10  <10
                         31           nan  nan   <10  <10  <10  nan
                         32           nan  nan   nan  nan  <10  nan
                         33           nan  nan   <10  nan  nan  nan
                         34           nan  nan   <10  nan  nan  nan
                         36           nan  nan   <10  nan  nan  nan
                         38           nan  nan   <10  nan  nan  nan
East of England          13           nan  nan   <10  nan  nan  nan
                         15           nan  nan   <10  nan  nan  nan
                         16           nan  nan   <10  nan  nan  nan
                         17           nan  nan   <10  nan  <10  nan
                         18           nan  nan   <10  <10  nan  nan
                         19           nan  nan   <10  nan  <10  nan
                         20           nan  nan   <10  <10  nan  nan
                         21           nan  nan   <10  nan  <10  nan
                         22           nan  nan   <10  <10  <10  nan
                         23           nan  nan   <10  <10  <10  nan
                         24           nan  nan   <10  <10  nan  nan
                         25           <10  nan   <10  <10  nan  nan
                         26           nan  nan   <10  nan  nan  nan
                         27           nan  nan   <10  <10  nan  nan
                         28           nan  nan   <10  <10  nan  nan
                         29           nan  nan   <10  <10  nan  nan
                         30           nan  nan   <10  nan  nan  nan
                         31           nan  nan   <10  <10  nan  nan
                         32           nan  nan   <10  nan  nan  nan
                         33           nan  nan   <10  nan  nan  nan
                         34           nan  nan   <10  <10  nan  nan
                         38           nan  nan   <10  nan  nan  nan
London                   13           nan  nan   <10  <10  nan  nan
                         15           nan  nan   <10  <10  nan  nan
                         16           nan  nan   <10  nan  <10  nan
                         17           nan  nan   <10  <10  nan  nan
                         18           nan  nan   <10  <10  nan  nan
                         19           nan  nan   <10  <10  <10  nan
                         20           nan  nan   <10  <10  <10  <10
                         21           nan  nan   <10  <10  nan  nan
                         22           nan  nan   <10  <10  nan  nan
                         23           nan  nan  16.0  <10  <10  nan
                         24           nan  nan  12.0  <10  <10  nan
                         25           nan  nan   <10  <10  <10  nan
                         26           nan  <10  11.0  <10  nan  nan
                         27           nan  nan   <10  <1

# Budesonide prescriptions by month

In [18]:
supressed_bd_prescriptions_by_month = suppress(df.groupby([pd.Grouper(key='first_positive_test_date',freq='M'),pd.Grouper(key='budesonide_prescription')]).count()['patient_id'].reset_index().rename(columns={"budesonide_prescription":"Count of budesonide prescriptions","patient_id":"Count of patients","first_positive_test_date":"Month of first positive test"}),['Count of patients'])
supressed_bd_prescriptions_by_month = pd.concat([supressed_bd_prescriptions_all.rename(columns={'cohort':'Month of first positive test'}),supressed_bd_prescriptions_by_month],ignore_index=True)
supressed_bd_prescriptions_by_month.style.hide_index()

Count of budesonide prescriptions,Count of patients,Month of first positive test
-2,<10,All
-1,10,All
0,737,All
1,165,All
2,71,All
3,15,All
0,<10,2021-06-30 00:00:00
0,<10,2021-07-31 00:00:00
0,<10,2021-08-31 00:00:00
0,<10,2021-09-30 00:00:00
